In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import pickle

In [2]:
df = pd.read_csv('Taxi_Trips_-_2021.csv')

In [3]:
df.dropna(subset=["Pickup Centroid Location", 
                  "Dropoff Centroid  Location", 
                  "Trip Start Timestamp", 
                  "Trip End Timestamp",
                  "Trip Seconds", 
                  "Trip Miles", 
                  "Company",
                  "Trip Total"
                  ""   ], 
          how="any", 
          inplace=True)

In [4]:
df

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Extras,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location
2,4bcfcfa78f372b705214ee2cba9c765ad6ba5161,31261f6e7fc645eff98c7964c7ea71a0ea7e387a6bd7f8...,01/01/2021 12:00:00 AM,01/01/2021 12:00:00 AM,120.0,0.10,NaN,NaN,8.0,8.0,...,1.0,4.50,Cash,Taxi Affiliation Services,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),41.899602,-87.633308,POINT (-87.6333080367 41.899602111)
3,655bbd9bcb0a78ee003e304ebbe350b21cdd7083,f6138aa35e1fb074eb79fbe29f512b6153dca3593d8cd2...,01/01/2021 12:00:00 AM,01/01/2021 12:00:00 AM,0.0,0.00,NaN,NaN,67.0,67.0,...,0.0,3.25,Cash,"Taxicab Insurance Agency, LLC",41.775929,-87.666596,POINT (-87.6665962653 41.7759288274),41.775929,-87.666596,POINT (-87.6665962653 41.7759288274)
4,7b7b8108de94a518607068a7369ceea1ab2ae924,2c508057a94474215bdea7b101edad0dc911ef61bb99e6...,01/01/2021 12:00:00 AM,01/01/2021 12:00:00 AM,50.0,0.38,NaN,NaN,60.0,31.0,...,0.0,4.00,Cash,Flash Cab,41.836150,-87.648788,POINT (-87.6487879519 41.8361501547),41.850266,-87.667569,POINT (-87.667569312 41.8502663663)
6,855259d1bc129f6370f038aa910067972eeae037,0aba795cd29b34ff868043bc923a341f37c4f6d7afabe4...,01/01/2021 12:00:00 AM,01/01/2021 12:15:00 AM,900.0,0.30,NaN,NaN,28.0,22.0,...,0.0,18.25,Cash,Taxi Affiliation Services,41.874005,-87.663518,POINT (-87.6635175498 41.874005383),41.922761,-87.699155,POINT (-87.6991553432 41.9227606205)
7,88194fab0a0a28df1b87e9f358143197d004d4d9,e11ecc8e74a2d3cbd2e9f58151ed09c81587cb491c50f6...,01/01/2021 12:00:00 AM,01/01/2021 12:15:00 AM,1140.0,4.30,NaN,NaN,6.0,4.0,...,0.0,15.25,Cash,Taxi Affiliation Services,41.944227,-87.655998,POINT (-87.6559981815 41.9442266014),41.975171,-87.687516,POINT (-87.6875155152 41.9751709433)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948037,dd96d78cedfcdac1d79e5d5929003d4a898e5a09,8a2802b8001fa243450b71eb66e6e76b4bdc4ea50cbb19...,12/31/2021 11:45:00 PM,01/01/2022 12:15:00 AM,2160.0,0.00,NaN,NaN,3.0,3.0,...,0.0,15.00,Cash,Taxi Affiliation Services,41.965812,-87.655879,POINT (-87.6558787862 41.96581197),41.965812,-87.655879,POINT (-87.6558787862 41.96581197)
3948039,ebe0df1d82f9ea4371b49525639352a12d5e9307,b3db2338ef61da4928f751b253fc10fcb116717e299cd4...,12/31/2021 11:45:00 PM,12/31/2021 11:45:00 PM,0.0,0.00,NaN,NaN,8.0,8.0,...,0.0,3.25,Cash,Taxi Affiliation Services,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),41.899602,-87.633308,POINT (-87.6333080367 41.899602111)
3948042,fdfaa0c7385e64942a51fc392d65b783d1fe3f2f,7e179f8ef66ae99ec2d1ec89224e0b7ee5469fe5627f6d...,12/31/2021 11:45:00 PM,01/01/2022 12:30:00 AM,3027.0,4.63,NaN,NaN,5.0,2.0,...,0.0,26.25,Prcard,Flash Cab,41.947792,-87.683835,POINT (-87.6838349425 41.9477915865),42.001571,-87.695013,POINT (-87.6950125892 42.001571027)
3948043,a9e0681fc38df3270c02d1352218a54e46a05463,c8f52a1c0e95bab6140b35851d62958e15d5032e622db7...,12/31/2021 11:45:00 PM,01/01/2022 12:15:00 AM,2400.0,12.70,1.703198e+10,1.703108e+10,56.0,8.0,...,4.0,51.35,Credit Card,"Taxicab Insurance Agency, LLC",41.785999,-87.750934,POINT (-87.7509342894 41.785998518),41.895033,-87.619711,POINT (-87.6197106717 41.8950334495)


In [5]:
features = ["Trip Start Timestamp", "Trip Seconds", "Trip Miles", "Company", 
            "Pickup Centroid Latitude", "Pickup Centroid Longitude", 
            "Dropoff Centroid Latitude", "Dropoff Centroid Longitude",
            "Trip Total"]

In [6]:
idx = [i for i,_ in enumerate(list(df["Company"].unique()))]
comp = list(df["Company"].unique())
company_dict = {comp[i]: idx[i] for i in idx} 
df["Company"] = [company_dict[i] for i in df["Company"]] 
am_pm = {"AM":"1", "PM":"2"}
df["Trip Start Timestamp"] = [int(t.split(' ')[-2].replace(":",'')+am_pm[t.split(' ')[-1]]) 
                                  for t in df["Trip Start Timestamp"]]

In [7]:
df_feature = df[features]

In [8]:
x = np.array(df_feature.iloc[:, :-1])
y = np.array(df_feature.iloc[:, -1:])

In [10]:
x_train, x_eval, y_train, y_eval = train_test_split(
                x, y, 
                test_size=0.2, 
                random_state=5
        )

In [20]:
model = LinearRegression()

In [22]:
model.fit(x_train, y_train)

LinearRegression()

In [23]:
model.score(x_eval, y_eval)

0.044466385634025274

In [31]:
y_predict = model.predict(x_train)
root_mean_square = (np.sqrt(mean_squared_error(y_train, y_predict )))
r2 = r2_score(y_train, y_predict)

In [32]:
print("rmse for training dataset: ", rmse)
print(f"acc is {acc}")

acc is 0.0377948298252172


In [44]:
y_predict_eval = model.predict(x_eval)
root_mean_square_eval = np.sqrt(mean_squared_error(y_predict_eval, y_eval))
r2_eval = r2_score(y_eval, y_predict_eval)

In [45]:
print("rmse for eval dataset: ", root_mean_square_eval)
print("r3 for eval: ", r2_eval)

rmse for eval dataset:  63.01452057598779
r3 for eval:  0.044466385634025274


In [47]:
pickle.dump(model, open("chicago_taxi.bin",  "wb"))

In [48]:
#----

In [50]:
model_ = pickle.load(open("chicago_taxi.bin", "rb"))
model_.score(x_eval, y_eval)

0.044466385634025274